In [3]:
import os
from re import S
import numpy as np
import torch
import torch.nn as nn
from torch import Tensor
import librosa
import yaml

from model import RawNet



In [4]:
config_path = "model_config_RawNet.yaml"
model_path = "models/model_LA_CCE_100_64_0.0001/epoch_71.pth"
device = 'cpu' 
with open(config_path, 'r') as f_yaml:
    parser1 = yaml.safe_load(f_yaml)

model = RawNet(parser1['model'], device).to(device)
model.load_state_dict(torch.load(model_path,map_location=device))

<All keys matched successfully>

In [5]:
def pad(x, max_len=64600):
    x_len = x.shape[0]
    if x_len >= max_len:
        return x[:max_len]
    # need to pad
    num_repeats = int(max_len / x_len)+1
    padded_x = np.tile(x, (1, num_repeats))[:, :max_len][0]
    return padded_x

def parse_input(file_path):
    cut = 64600  # take ~4 sec audio (64600 samples)
    X, fs = librosa.load(file_path, sr=16000)
    X_pad = pad(X, cut)
    x_inp = Tensor(X_pad)
    return x_inp.unsqueeze(0).to(device)

In [6]:
model.eval()
# Load the audio file that we need to check whether it is fake voice or not.
wav = "/root/dataset/Dataset/ASVspoof/LA/ASVspoof2019_LA_eval/flac/LA_E_5085671.flac"
# wav = "/root/dataset/ASVspoof/ASVspoof2021_LA_eval/flac/LA_E_5464494.flac"
# wav = "/root/dataset/ASVspoof/ASVspoof2021_LA_eval/flac/LA_E_2925019.flac"

# wav = "/root/dataset/Speech/AIHub_Studio/wavs/01FHSH0028/01FHSH0028_00122.wav"

# wav = "/root/biological/asvspoof2021/LA/Baseline-RawNet2-bio/Demo/prof_16k.flac"

# Extract the feature of the audio

x_inp = parse_input(wav)

# use the model to calculate the prediction value
# out, _ = model(x_inp)
out, _ = model(x_inp)

# out = [0.123345135,-0.1234145124]

# Convert into readable format
_, pred = out.max(dim=1)
if pred:
    print("bonafide") #real voice
else:
    print("spoof") #fake voice

spoof


# Android Conversion

In [7]:
import torch
from torch import Tensor
from torch.utils.mobile_optimizer import optimize_for_mobile


In [8]:
class BTSDetect(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
    
    def forward(self, wavforms: Tensor):
        logits, _ = self.model(wavforms)
        _, pred = logits.max(dim=1)
        if pred:
            return 0 # bonafide
        else:
            return 1 # fake

model = BTSDetect(model)


In [9]:
# # Apply quantization / script / optimize for motbile

quantized_model = torch.quantization.quantize_dynamic(
    model, qconfig_spec={torch.nn.Linear}, dtype=torch.qint8)
scripted_model = torch.jit.script(quantized_model)
optimized_model = optimize_for_mobile(scripted_model)


# Sanity check

print('Result:', optimized_model(x_inp))
optimized_model._save_for_lite_interpreter("btsdetect.ptl")

Result: 1
